# 📰 POS and NER Practical Example on Realistic Text Data (NLP)

This notebook demonstrates a **complete practical workflow** applying:

- Text preprocessing
- Part-of-Speech (POS) tagging
- Named Entity Recognition (NER)

on **real-world BBC news data** using:
- spaCy
- pandas
- NLTK
- regular expressions

The goal is to move beyond toy examples and apply NLP techniques
to **realistic, noisy text data**.


## 1️⃣ Overview of the Workflow

In this notebook, we:

1. Load real BBC news data
2. Focus on the **news titles** only
3. Clean and preprocess text:
   - lowercasing
   - stopword removal
   - punctuation removal
   - tokenization
   - lemmatization
4. Apply **POS tagging** using spaCy
5. Analyze most frequent nouns, verbs, and adjectives
6. Apply **Named Entity Recognition (NER)**
7. Visualize detected entities using displaCy


## 2️⃣ Importing Libraries

We use a combination of NLP and data analysis libraries:

- **NLTK** for tokenization, stopwords, and lemmatization
- **spaCy** for POS tagging and NER
- **pandas** for structured data manipulation
- **regex** for text cleaning
- **matplotlib** for optional visualization


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re
import pandas as pd
import matplotlib.pyplot as plt
from spacy import displacy 
from IPython.display import HTML, display

## 3️⃣ Loading the BBC News Dataset

We load BBC news data from a CSV file.
Each row represents a news article, including a title and content.

For this exercise, we focus only on the **title column**,
which is short but rich in linguistic information.


# Load Data

In [ ]:
bbc_data = pd.read_csv('../../data/bbc_news.csv')

In [ ]:
bbc_data.head()

In [ ]:
bbc_data.info()  

## 4️⃣ Selecting the Text for Analysis

To simplify the analysis and reduce noise, we extract
only the **news titles** into a separate DataFrame.


In [ ]:

titles = pd.DataFrame(bbc_data['title'])
titles.head()

## 5️⃣ Text Cleaning and Preprocessing

Before applying POS tagging and NER, we clean the text to improve consistency.

Steps applied:
- convert text to lowercase
- remove stopwords
- remove punctuation
- tokenize text
- lemmatize tokens

This preprocessing helps reduce noise and normalize words
before linguistic analysis.


In [ ]:
titles['lowercase'] = titles['title'].str.lower()
titles.head()

In [ ]:
en_stopwords = stopwords.words('english')
titles['no_stopwords'] = titles['lowercase'].apply(lambda x: ' '. join([word for word in x.split() if word not in (en_stopwords)]))
titles.head()

In [ ]:
#punctuation removal
titles['no_stopwords_nopunct'] = titles.apply(lambda x: re.sub(r"[^\w\s]", ' ', x['no_stopwords']), axis=1)
titles.head()

In [ ]:
# tokenize
titles['tokens_raw'] = titles.apply(lambda x: word_tokenize(x['title']) , axis=1)
titles['tokens_clean'] = titles.apply(lambda x: word_tokenize(x['no_stopwords_nopunct']) , axis=1)

In [ ]:
#lemmatization
lemmatizer = WordNetLemmatizer()
titles['tokens_clean_lemmatized'] = titles['tokens_clean'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
titles.head()

## 6️⃣ Preparing Tokens for spaCy

spaCy expects text input as a string, not a list of lists.

After tokenizing and lemmatizing each title, we:
- flatten all token lists
- join them into a single string

This allows spaCy to process the entire dataset in one pass.


In [ ]:
tokens_raw_list = sum(titles['tokens_raw'], [])   
tokens_clean_list = sum(titles['tokens_clean_lemmatized'], []) 

## 7️⃣ Part-of-Speech (POS) Tagging

Using spaCy, we assign a grammatical role
(noun, verb, adjective, etc.) to each token.

The result is stored in a pandas DataFrame with:
- the token
- its POS tag


In [ ]:

nlp = spacy.load("en_core_web_sm")

In [ ]:
spacy_doc = nlp(' '.join(tokens_clean_list))  

In [ ]:
records = [(t.text, t.pos_) for t in spacy_doc]
pos_df = pd.DataFrame(records, columns=['token', 'pos_tag'])
pos_df.head()

## 8️⃣ Analyzing POS Frequencies

We group tokens by:
- word
- POS tag

and count how often each combination appears.

This allows us to:
- identify common nouns
- identify frequent verbs
- identify descriptive adjectives


In [ ]:
pos_df_counts = pos_df.value_counts(['token', 'pos_tag']).reset_index(name='counts')


In [ ]:
pos_df_counts.head(10)

## 9️⃣ Extracting Top POS Categories

From the POS frequency table, we extract:
- the most common nouns
- the most common verbs
- the most common adjectives

This provides insight into the **language style**
of BBC news headlines.


In [ ]:
nouns = pos_df_counts[pos_df_counts['pos_tag'] == 'NOUN'][0:10]
nouns

In [ ]:
verbs = pos_df_counts[pos_df_counts['pos_tag'] == 'VERB'][0:10]
verbs

In [ ]:
adjectives = pos_df_counts[pos_df_counts['pos_tag'] == 'ADJ'][0:10]
adjectives

## 🔟 Named Entity Recognition (NER)

Next, we apply **Named Entity Recognition** to detect:
- people
- organizations
- locations
- dates
- geopolitical entities

spaCy automatically identifies these entities
based on context and linguistic cues.


In [ ]:
records = []
for ent in spacy_doc.ents:
    records.append((ent.text, ent.label_))
ner_df = pd.DataFrame(records, columns=['token', 'ner_tag'])
ner_df.head()

## 1️⃣1️⃣ Counting Named Entities

We count how often each named entity appears
and group them by:
- entity text
- entity label

This helps identify:
- frequently mentioned organizations
- recurring locations
- prominent people in the news


In [ ]:
ner_df_counts = ner_df.value_counts(['token', 'ner_tag']).reset_index(name='counts')
ner_df_counts.head(10)


## 1️⃣2️⃣ Visualizing Named Entities with displaCy

Finally, we use **displaCy** to visually inspect
the named entities detected by spaCy.

Entities are highlighted directly in the text
using different colors for each label.


In [ ]:


html = displacy.render(spacy_doc, style="ent", jupyter=False)  
display(HTML(html))  

## ✅ Final Takeaways

- Real-world text requires careful preprocessing
- POS tagging reveals grammatical structure
- NER extracts meaningful real-world information
- spaCy makes advanced NLP tasks accessible
- Visualization helps validate model behavior
- Understanding the pipeline is more important than copying code
